In [1]:
%matplotlib inline
import numpy as np
import scipy as sp
import os
import matplotlib.pylab as plt
from sys import getsizeof
from pandas import DataFrame
import scipy.signal
from split_safe import SafeDataFilter
from sklearn.preprocessing import StandardScaler
from data_scaler import scale_across_features, scale_across_time

/Users/Anuar_The_Great/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
X_train1 = np.load('/Users/Anuar_The_Great/desktop/ML/CNN/train_1_npy/X_new.npy')
X_train2 = np.load('/Users/Anuar_The_Great/desktop/ML/CNN/train_2_npy/X_new.npy')
X_train3 = np.load('/Users/Anuar_The_Great/desktop/ML/CNN/train_3_npy/X_new.npy')
y_train1 = np.load('/Users/Anuar_The_Great/desktop/ML/CNN/train_1_npy/y_new.npy')
y_train2 = np.load('/Users/Anuar_The_Great/desktop/ML/CNN/train_2_npy/y_new.npy')
y_train3 = np.load('/Users/Anuar_The_Great/desktop/ML/CNN/train_3_npy/y_new.npy')

X_test1 = np.load('/Users/Anuar_The_Great/desktop/ML/CNN/test_1_new/X_new.npy')
X_test2 = np.load('/Users/Anuar_The_Great/desktop/ML/CNN/test_2_new/X_new.npy')
X_test3 = np.load('/Users/Anuar_The_Great/desktop/ML/CNN/test_3_new/X_new.npy')



X_train1[X_train1 < -100000] = 0
X_train2[X_train2 < -100000] = 0
X_train3[X_train3 < -100000] = 0

X_test1[X_test1 < -100000] = 0
X_test2[X_test2 < -100000] = 0
X_test3[X_test3 < -100000] = 0

X_train1 = np.abs(X_train1)
X_train2 = np.abs(X_train2)
X_train3 = np.abs(X_train3)

X_test1 = np.abs(X_test1)
X_test2 = np.abs(X_test2)
X_test3 = np.abs(X_test3)

X_trains = [X_train1, X_train2, X_train3]
X_tests = [X_test1, X_test2, X_test3]
y_trains = [y_train1, y_train2, y_train3]
pos_weight = []
    
# Accepts X's of shape (examples, channels, time-windows, bins)
# X_trains and X_tests must be list
def scale_reshape(X_trains, X_tests):
    # applying scale_across_time
    for i, X in enumerate(X_trains):
        X_train = X.swapaxes(2, 3)
        X_test = X_tests[i]
        X_test = X_test.swapaxes(2, 3)
        X_trains[i], scalers = scale_across_time(X_train, X_test)
        X_tests[i], additional_scalers = scale_across_time(X_test, scalers=scalers)
    # swaping the axis back and reshaping the features
    for i, X in enumerate(X_trains):
        X_trains[i] = X.swapaxes(2, 3)
        X_trains[i] = X.reshape((X.shape[0], 10, 112))   
        X = X_tests[i]
        X_tests[i] = X.swapaxes(2, 3)
        X_tests[i] = X.reshape((X.shape[0], 10, 112)) 
    return X_trains, X_tests, scalers
    
X_trains, X_tests, scalers = scale_reshape(X_trains, X_tests)
# Computing y's one-hot vectors and pos_weight
for i, X in enumerate(X_trains):
    ys = np.zeros((y_trains[i].shape[0], 2))
    ys[:, 1] = (y_trains[i] > 0).reshape(y_trains[i].shape[0],)
    ys[:, 0] = (y_trains[i] < 1).reshape(y_trains[i].shape[0],)
    y_trains[i] = ys
    pos_weight.append(np.sum(y_trains[i][:,0])/np.sum(y_trains[i][:,1]))
    

# Checking:
print(np.nonzero(X_trains[0][X_trains[0] > 50]))
print(np.nonzero(X_trains[0][X_trains[0] < -5]))
print("final: ", X_trains[0].shape)
print("final: ", y_trains[0].shape)

(719, 16, 7, 10)
(216, 16, 7, 10)
(1986, 16, 7, 10)
(1002, 16, 7, 10)
(2058, 16, 7, 10)
(690, 16, 7, 10)
(array([    0,     1,     2, ..., 28802, 28803, 28804]),)
(array([], dtype=int64),)
('final: ', (719, 10, 112))
('final: ', (719, 2))


In [3]:
from keras import backend as K
from keras.engine.topology import Layer

class GlobalPoolLayer(Layer):
    def __init__(self, **kwargs):
        #self.output_dim = output_dim
        super(GlobalPoolLayer, self).__init__(**kwargs)
        
    def build(self, input_shape):
        #self.built = True
        super(GlobalPoolLayer, self).build()
        
    def call(self, x, mask=None):
        x += 1e-06
        avg = x.mean(3, dtype='float32')
        max_val = x.max(3)
        min_val = x.min(3)
        var = x.var(3)
        geom_mean = K.exp(K.mean(K.log(x), axis=3, dtype='float32'))
        l2_norm = x.norm(2, axis=3)
        output = K.concatenate([avg, max_val, min_val, var, geom_mean, l2_norm], axis=2)
        return output
    
    def get_output_shape_for(self, input_shape):
        return (input_shape[0], 6)

Using Theano backend.


Couldn't import dot_parser, loading of dot files will not be possible.


In [4]:
#from __future__ import print_function
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Reshape
from keras.layers import Convolution1D
from sklearn.grid_search import GridSearchCV, RandomizedSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.metrics import roc_auc_score
from numpy.random import randint, random
from keras.regularizers import l2, activity_l2
from keras.callbacks import EarlyStopping



# create your model using this function
def create_model(nb_filter1=16, nb_filter2=32, activation1='relu', l2_weight1=0.0, l2_weight2=0.0,
                 dropout_rate=0.3, optimizer='adam', nb_epoch=10, hidden_dims=112):
    #batch_size = 100
    filter_length = 1
    n = X_train.shape[0]
    
    print('Build model...')
    model = Sequential()
    model.add(Convolution1D(nb_filter=nb_filter1,
                            filter_length=filter_length,
                            init='glorot_normal',
                            border_mode='valid',
                            activation=activation1,
                            subsample_length=1,
                            W_regularizer=l2(l2_weight1),
                            input_shape=(10, 112)))
    
    model.add(Convolution1D(nb_filter=nb_filter2,
                            filter_length=1,
                            init='glorot_normal',
                            border_mode='valid',
                            subsample_length=1,
                            W_regularizer=l2(l2_weight2),
                            activation=activation1))    

    model.add(Reshape((nb_filter2*10,)))
    model.add(Dense(hidden_dims))
    model.add(Activation(activation1))
    model.add(Dropout(dropout_rate))
    #model.add(GlobalPoolLayer())
    #model.add(Dense(hidden_dims))
    #model.add(Activation('sigmoid'))
    model.add(Dense(2))
    model.add(Activation('softmax'))
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['binary_accuracy'])
    #model.summary()
    return model

In [ ]:
model = create_model(16, 32, 'sigmoid', 0, 0, 0.3, 'adam', 20, 36)
model_result = model.fit(X_trains[2], y_trains[2], class_weight={0: 1.0, 1: pos_weight[2]}, verbose=2, nb_epoch=100)

preds = model_result.model.predict_classes(X_trains[2])
#preds_submission = np.argmax(preds, axis=1).astype('int32')

print("\n")
print(roc_auc_score(y_trains[2][:, 1], preds))